In [1]:
import lcls_live
import os
from lcls_live.datamaps import get_datamaps
from lcls_live.archiver import lcls_archiver_restore

from pytao import Tao
import pandas as pd

from copy import deepcopy

In [2]:
# Pick a model and slice

MODEL = 'sc_bsyd' # or sc_diag0 or sc_inj

BEGELE = 'BEGINNING'
#ENDELE = 'OTR0H04'
#ENDELE = 'ENDCOL0'
ENDELE = 'END'

tao = Tao(f'-noplot -init $LCLS_LATTICE/bmad/models/{MODEL}/tao.init -slice {BEGELE}:{ENDELE}')

def ele_info(ele):
    dat = tao.ele_head(ele)
    dat.update(tao.ele_gen_attribs(ele))
    return dat

def ele_table(match="*"):
    ix_ele = tao.lat_list(match, "ele.ix_ele", flags="-no_slaves")
    dat = list(map(ele_info, ix_ele))
    df = pd.DataFrame(dat, index=ix_ele)
    df.L.fillna(0, inplace=True)
    df['s_center'] = df['s'] - df['L']/2
    df['s_beginning'] = df['s'] - df['L']
    return  df

df = ele_table()
good_names = set(df['name'])


In [3]:
def filter_datamap(dm, bmad_names):
    bnames = dm.data['bmad_name'] 
    bmad_names = set(bmad_names)
    ix = bnames[[name in bmad_names for name in bnames]].index
    dm2 = deepcopy(dm)
    dm2.data = dm.data.loc[ix]
    return dm2

def tao_commands(pvdata):
    cmds = []
    for name, dm in DM.items():
        cmds.extend(dm.as_tao(pvdata))
    return cmds

def save_cmds(cmds, filename='cmds.tao'): # Write to file for running with vanilla Tao
    with open(filename, 'w') as f:
        f.write('set global lattice_calc_on = F\n')
        f.write('set global plot_on = F\n')    
        f.write('set ele quad::* field_master = T\n')
        f.write('set ele SC_L* is_on = F\n')
        for cmd in cmds:
            f.write(cmd+'\n')
        f.write('set global lattice_calc_on = T\n')        
        f.write('set global plot_on = T\n')   

In [5]:
DM0 = get_datamaps(MODEL)
DM = {}
for name, dm in DM0.items():
    print(type(dm))
    if name == 'tao_energy_measurements':
        DM[name] = dm
    else:
        DM[name] = filter_datamap(dm, good_names)

# datamaps to exclude
DENYLIST = [
    #'bpms',
   # 'cavities',
    'correctors',  # correctors account for errors that we do not know about.
    #'quad',
]

#Filter out data maps in denylist
PVLIST =  []
for name, dm in DM.items():
    if name in DENYLIST:
        continue
    PVLIST.extend(dm.pvlist)
PVLIST = list(set(PVLIST))
print('Number of PVs to be polled: {}'.format(len(PVLIST)))

<class 'lcls_live.datamaps.tabular.TabularDataMap'>
<class 'lcls_live.datamaps.tabular.TabularDataMap'>
<class 'lcls_live.datamaps.tabular.TabularDataMap'>
<class 'lcls_live.datamaps.tabular.TabularDataMap'>
<class 'lcls_live.datamaps.tabular.TabularDataMap'>
<class 'lcls_live.datamaps.tabular.TabularDataMap'>
<class 'lcls_live.datamaps.tabular.TabularDataMap'>
Number of PVs to be polled: 1103


In [10]:
# How to open an epics tunnel through port 8080
# ssh -D 8080 -Y <USER_ID>@mcclogin.slac.stanford.edu -J <USER_ID>@centos7.slac.stanford.edu

epics_timestamp = '2023-07-17T19:45:00.000000-07:00'

os.environ['http_proxy']='socks5h://localhost:8080'
os.environ['HTTPS_PROXY']='socks5h://localhost:8080'
os.environ['ALL_PROXY']='socks5h://localhost:8080'

PVDATA=lcls_archiver_restore(PVLIST, epics_timestamp)

Requesting: http://lcls-archapp.slac.stanford.edu/retrieval/data/getDataAtTime?at=2023-07-17T19:45:00.000000-07:00&includeProxies=true


In [11]:
cmds=tao_commands(PVDATA)
save_cmds(cmds,filename='pvs_{}_{}.tao'.format(MODEL,epics_timestamp))